In [1]:
import os
import pdfplumber
from collections import defaultdict

def detect_columns(text_blocks, page_width):
    """Gruppiere Textblöcke basierend auf ihrer horizontalen Position (Spalten)."""
    columns = defaultdict(list)
    threshold = page_width * 0.15  # Threshold, um Spalten zu erkennen

    for block in text_blocks:
        x_center = (block['x0'] + block['x1']) / 2

        assigned = False
        for col_x in columns:
            if abs(col_x - x_center) < threshold:
                columns[col_x].append(block)
                assigned = True
                break
        
        if not assigned:
            columns[x_center].append(block)

    return columns

def extract_text_from_column(column_blocks):
    """Sortiere die Blöcke innerhalb einer Spalte und extrahiere den Text."""
    sorted_blocks = sorted(column_blocks, key=lambda block: (block['top'], block['x0']))
    return ' '.join([block['text'] for block in sorted_blocks])

def extract_text_from_page(page):
    """Extrahiere den Text aus einer Seite, behalte dabei das Layout."""
    text_blocks = page.extract_words()
    if not text_blocks:
        return ""

    columns = detect_columns(text_blocks, page.width)
    full_text = []

    for col_x, blocks in sorted(columns.items()):
        column_text = extract_text_from_column(blocks)
        full_text.append(column_text)

    return "\n\n".join(full_text)

def extract_text_from_pdfs(pdf_folder_path):
    documents = []

    for filename in os.listdir(pdf_folder_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_folder_path, filename)

            with pdfplumber.open(pdf_path) as pdf:
                full_text = []

                for page in pdf.pages:
                    page_text = extract_text_from_page(page)
                    full_text.append(page_text)

                documents.append("\n\n".join(full_text))

    return documents

# Pfad zum PDF-Ordner
pdf_folder_path = os.path.join('data', 'pdf', 'g1')

# Extrahiere die Texte aus den PDFs
documents = extract_text_from_pdfs(pdf_folder_path)

# Beispielhafte Ausgabe des ersten Dokuments
print(documents[0] if documents else "Keine Dokumente gefunden")

Der blaue Altglascontainer. Das kommt hinein Saubere Flaschen und Deckelgläser nach den Farben Weiß, Grün und Braun getrennt; blaues oder buntes Glas bitte zum Grünglas geben.  Glasfl aschen  Konservengläser Das kommt nicht hinein  Flachglas wie Spiegel und Scheiben (Restmüll oder  Trinkgläser und Blumenvasen (Restmüll)  Porzellan und Keramik (Restmüll)  Glühbirnen (Restmüll), Energiesparlampen (Schadstoffmobil oder Wertstoffhöfe)  Stark verschmutzte Gläser (Restmüll) Standort: auf öffentlichen

immer Sperrmüll) Plätzen

Was kommt wo hinein? FES Frankfurter Entsorgungs- Abfall richtig trennen – und Service GmbH Weidenbornstraße 40 60389 Frankfurt am Main Servicetelefon 0800 2008007-0 Servicetelefax 069 212-31323 services@fes-frankfurt.de www.fes-frankfurt.de FES-Servicecenter Liebfrauenberg 52–54 (Ecke Töngesgasse/Schärfengässchen) 60313 Frankfurt am Main Mo. – Fr. 10.00 – 18.00 Uhr Sa. 10.00 – 16.00 Uhr rgungsfachbetrie Entso b rtschafte.V.-EdDn oE El -öK tsn 0 gla3 sn1 we4 rge

In [2]:
import re

def clean_text(documents):
    """Durchläuft die Liste der Dokumente und bereinigt die Texte."""
    cleaned_documents = []
    
    # Liste unerwünschter Zeichen oder Muster
    unwanted_patterns = [
        r"",     # Entferne das Symbol 
        r"",     # Beispiel für ein weiteres unerwünschtes Symbol
        r"(\b\w+\s*-\s*\w+\b)",  # Entferne Worte mit unvollständigen Endungen (z.B. "Glasfl aschen")
        r"\n+",   # Entferne überflüssige Zeilenumbrüche
        r"\s{2,}",# Entferne überflüssige Leerzeichen
        r"\s*:\s*",  # Entferne überflüssige Doppelpunkte
        r"\b\w{1,2}\b",  # Entferne isolierte kurze Wörter (z.B. "fi" in "finden")
        r"[^A-Za-z0-9äöüÄÖÜß.,;:!?()\-'\s]"  # Entferne nicht-alphanumerische Zeichen
    ]

    # Regex-Muster kombinieren
    combined_pattern = re.compile("|".join(unwanted_patterns))

    for document in documents:
        # Entferne unerwünschte Zeichen und Muster
        cleaned_text = combined_pattern.sub(" ", document)
        # Entferne mehrfach aufeinanderfolgende Leerzeichen, um den Text zu bereinigen
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        cleaned_documents.append(cleaned_text)
    
    return cleaned_documents

# Verwende die bestehende documents-Liste aus dem vorherigen Skript
cleaned_documents = clean_text(documents)

# Beispielhafte Ausgabe eines bereinigten Dokuments
print(cleaned_documents[2])

NameError: name 'documents' is not defined

In [5]:
import re
import spacy

# Lade das deutsche spaCy-Modell
nlp = spacy.load('de_core_news_sm')

def preprocess_text_with_advanced_cleaning(documents):
    """Bereinigt die Dokumente, entfernt unnötige Stopwörter und normalisiert den Text."""
    custom_stopwords = nlp.Defaults.stop_words.difference({'nicht', 'kein', 'wichtig'})

    preprocessed_documents = []

    for document in documents:
        text = document.lower()
        text = re.sub(r'\b(?:[\w\-]+\.)+[\w]{2,4}(?:/[\w\-.?%&=]*)?\b', '', text)
        text = re.sub(r'\b[\w\.\-]+@[\w\.\-]+\.\w{2,4}\b', '', text)
        text = re.sub(r'\b(?:www|servicetelefon|servicetelefax|services|frankfurt|main|weidenbornstraße|liebfrauenberg)\b', '', text)
        text = re.sub(r'[^a-zäöüß\s]', '', text)

        doc = nlp(text)
        filtered_tokens = [token.text for token in doc if token.text not in custom_stopwords and len(token.text) > 2]

        # Entferne unzusammenhängende Zeichenfolgen und extrem kurze Wörter
        text = ' '.join(filtered_tokens)
        text = re.sub(r'\b\w{1,3}\b', '', text)

        # Entferne mehrfach aufeinanderfolgende gleiche Wörter
        text = re.sub(r'\b(\w+)\s+\1\b', r'\1', text)

        # Entferne weiterhin sinnlose Textfragmente
        text = re.sub(r'\b[a-zäöüß]{2,3}\b\s+\b[a-zäöüß]{2,3}\b', '', text)

        preprocessed_documents.append(text.strip())

    return preprocessed_documents

# Bereinige die Dokumente weiter
preprocessed_documents = preprocess_text_with_advanced_cleaning(cleaned_documents)

# Beispielhafte Ausgabe eines bereinigten Dokuments
print(preprocessed_documents[2])

gelbe ausschließlich glas nicht verpackungen altkleider batterien akkus behälterglas blechgeschirr  disketten druckerpatronen einwegrasierer elektrogeräte essensreste faltschachteln feuerzeuge ilme dvds videokassetten energiesparlampen regionale nähere infos tonne gebrauchte restentleerte verpackungen verpackungsbestandteile bitte voneinander kunststoffdeckel arzneimittelblister butterfolie joghurtbecher einkaufstüten gemüsebeutel kunststoff eisverpackungen üllmaterial verpackungen luftpolsterfolie schaumstoff konservendosen kronkorken lebensmittel gelbe tonne papier pappe karton glas gummi holzwolle hygieneartikel katzenstreu keramikprodukte kinderspielzeug klarsichthüllen kugelschreiber nicht restentleerte papier pappe papiertaschentücher pflaster verbandsmaterial porzellan bioabfälle ausnahmen     wertstofftonne erhältst kommune klicken zuständige ansprechpartner gelben sack nicht trennen versand kunststoff gelben sämtliche abfälle verpackungen scannen gehören papier pappe karton au